### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session on your Ubuntu_Azure_VM:

* login (ssh) into the VM:
```
loginname1@Ubuntu_Azure_VM:/datadrive01/prj/regularR_Realtime$ jupyter notebook --notebook-dir=$(pwd) --ip='*' --port=9000 --no-browser --allow-root
```
  
   
* then go to:    
http://Ubuntu_Azure_VM.eastus2.cloudapp.azure.com:9000/ 
  
  
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes  
2019-03-11
Azure Machine Learning SDK for Python v1.0.18  
   
Note: As of 02/13/2019, latest versions are:  
Python 3.7.2 (https://www.python.org/downloads/, https://anaconda.org/anaconda/python)  
miniconda3 4.5.12 (https://hub.docker.com/r/continuumio/miniconda3/tags)  
AML SDK  v.1.0.15 (https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  
Ubuntu 19.04 (https://hub.docker.com/_/ubuntu?tab=tags)  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [3]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "aml-sdk_docker_image"
image_version = 'sdk.v1.0.60' # also set this in conda file below

Overwriting .env


In [4]:
%dotenv
docker_file_location = 'docker_build'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

In [5]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

#clean previous history (i.e. content of amlsdk/docker)
!mkdir -p {docker_file_dir}
!chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.60'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml'

total 0


In [6]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.6 # 3.6 req by tf, which seems to be req by automl, not 3.7.2 
  - numpy
  - cython
  - notebook 
  - nb_conda
  - scikit-learn
  - pip
  - pip:
    - python-dotenv
    - papermill[azure]
    - azureml-sdk[notebooks,automl,explain]==1.0.60 

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml


In [7]:
%%writefile $docker_file_path 

# FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly 
FROM continuumio/miniconda3:4.7.10 as AzureMLSDKOnly    
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gpg-agent \
    gcc g++ \
    wget bzip2 \
    curl apt-transport-https lsb-release gpg \
    git make \
    sudo && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME aml-sdk-conda-env    

# reate aml sdk conda env defined by .yml file
RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    /opt/conda/bin/conda clean  --yes --all 
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH                
        
FROM AzureMLSDKOnly as AzureMLSDKAndAzCli
# Install Azure CLI
ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $AZ_CLI_REPO main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 
    
# Install AzCopy
RUN mkdir -p /tmp/azcopy && mkdir -p /azcopy10 && \
    wget -O /tmp/azcopy/azcopyv10.tar.gz https://aka.ms/downloadazcopy-v10-linux &&  \
    tar -xf /tmp/azcopy/azcopyv10.tar.gz -C /azcopy10 
RUN rm -rf /tmp/azcopy
ENV PATH=/azcopy10/azcopy_linux_amd64_10.1.2:$PATH
    
FROM AzureMLSDKAndAzCli as AzureMLSDKAndAzCliAndAZDevOps
#https://marketplace.visualstudio.com/items?itemName=ms-vsts.cli
RUN az extension add --name azure-devops
        
CMD /bin/bash


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.60


In [8]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
aml_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
image_version_string
# aml_sudo_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
# 'sudo' + image_version_string
aml_azcli_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcli' + image_version_string
aml_azcli_azdevops_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcliazdevops'  + image_version_string


docker_file_path
aml_docker_image_name
# aml_sudo_docker_image_name
aml_azcli_docker_image_name
aml_azcli_azdevops_docker_image_name

docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.60'

'georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.60'

'georgedockeraccount/aml-sdk_docker_imageazcli:sdk.v1.0.60'

'georgedockeraccount/aml-sdk_docker_imageazcliazdevops:sdk.v1.0.60'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

total 8
-rw-rw-r-- 1 loginvm001 loginvm001  321 Sep 11 03:45 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm001 loginvm001 1840 Sep 11 03:45 dockerfile_aml-sdk_docker_image_sdk.v1.0.60


In [9]:
build_command = 'docker build -t ' + aml_docker_image_name  + \
 ' --target AzureMLSDKOnly -f ' + \
docker_file_path +' '+ working_path + ' --no-cache'
build_command

!     $build_command
!     docker build -t $aml_azcli_docker_image_name  --target AzureMLSDKAndAzCli -f $docker_file_path $working_path
!     docker build -t $aml_azcli_azdevops_docker_image_name -f $docker_file_path $working_path


'docker build -t georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.60 --target AzureMLSDKOnly -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.60 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build --no-cache'

Sending build context to Docker daemon  4.608kB
Step 1/11 : FROM continuumio/miniconda3:4.7.10 as AzureMLSDKOnly
4.7.10: Pulling from continuumio/miniconda3

bdfe9778: Pulling fs layer 
28bd8be5: Pulling fs layer 
Digest: sha256:54eb3dd4003f11f6a651b55fc2074a0ed6d9eeaa642f1c4c9a7cf8b148a30ceb
Status: Downloaded newer image for continuumio/miniconda3:4.7.10
 ---> 4a51de2367be
Step 2/11 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in abc3d425a913
Removing intermediate container abc3d425a913
 ---> 74f737b6c8ba
Step 3/11 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gpg-agent     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in 27339bae990c
Get:1 http://security.debian.org/debian-security buster/updates InRelease [39.1 kB]
Get:2 http://deb.debian.org/debian buster InRelease [122 kB]
Get:3 http://deb.debian.org/debi

debconf: delaying package configuration, since apt-utils is not installed
Fetched 52.4 MB in 1s (80.1 MB/s)
Selecting previously unselected package libpython3.7-minimal:amd64.
(Reading database ... 12557 files and directories currently installed.)
Preparing to unpack .../libpython3.7-minimal_3.7.3-2_amd64.deb ...
Unpacking libpython3.7-minimal:amd64 (3.7.3-2) ...
Selecting previously unselected package python3.7-minimal.
Preparing to unpack .../python3.7-minimal_3.7.3-2_amd64.deb ...
Unpacking python3.7-minimal (3.7.3-2) ...
Setting up libpython3.7-minimal:amd64 (3.7.3-2) ...
Setting up python3.7-minimal (3.7.3-2) ...
Selecting previously unselected package python3-minimal.
(Reading database ... 12797 files and directories currently installed.)
Preparing to unpack .../python3-minimal_3.7.3-1_amd64.deb ...
Unpacking python3-minimal (3.7.3-1) ...
Preparing to unpack .../bzip2_1.0.6-9.2~deb10u1_amd64.deb ...
Unpacking bzip2 (1.0.6-9.2~deb10u1) over (1.0.6-9.1) ...
Preparing to unpack .../

Selecting previously unselected package sudo.
Preparing to unpack .../41-sudo_1.8.27-1_amd64.deb ...
Unpacking sudo (1.8.27-1) ...
Setting up apt-transport-https (1.8.2) ...
Setting up distro-info-data (0.41) ...
Setting up binutils-common:amd64 (2.31.1-16) ...
Setting up linux-libc-dev:amd64 (4.19.67-2) ...
Setting up libnpth0:amd64 (1.6-1) ...
Setting up libassuan0:amd64 (2.5.2-1) ...
Setting up libgomp1:amd64 (8.3.0-6) ...
Setting up bzip2 (1.0.6-9.2~deb10u1) ...
Setting up libasan5:amd64 (8.3.0-6) ...
Setting up make (4.2.1-1.2) ...
Setting up libmpfr6:amd64 (4.0.2-1) ...
Setting up libquadmath0:amd64 (8.3.0-6) ...
Setting up libmpc3:amd64 (1.1.0-1) ...
Setting up libatomic1:amd64 (8.3.0-6) ...
Setting up sudo (1.8.27-1) ...
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libmpx2:amd64 (8.3.0-6) ...
Setting up libubsan1:amd64 (8.3.0-6) ...
Setting up libisl19:amd64 (0.20-2) ...
Setting up gpgconf (2.2.12-1+deb10u1) ..

Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
scipy-1.3.1-py36h7c811a0_0.tar.bz2          18.1 MB
ipython_genutils-0.2.0-py36_0.tar.bz2         39 KB
ipykernel-5.1.2-py36h39e3cac_0.tar.bz2       165 KB
libedit-3.1.20181209-hc058e9b_0.tar.bz2      188 KB
decorator-4.4.0-py36_1.tar.bz2                18 KB
nbconvert-5.5.0-py_0.tar.bz2                 381 KB
webencodings-0.5.1-py36_1.tar.bz2             19 KB
pygments-2.4.2-py_0.tar.bz2                  664 KB
six-1.12.0-py36_0.tar.bz2                     22 KB
ipython-7.8.0-py36h39e3cac_0.tar.bz2         1.1 MB
terminado-0.8.2-py36_0.tar.bz2                22 KB
jupyter_client-5.3.1-py_0.tar.bz2             69 KB
tk-8.6.8-hbc83047_0.tar.bz2                  3.1 MB
nb_conda_kernels-2.2.2-py36_0.tar.bz2         37 KB
nbformat-4.4.0-py36_0.tar.bz2                141 KB
mkl-service-2.3.0-py36he904b0f_0.tar.bz2     208 KB
prometheus_client-0.7.1-py_0.tar.bz2          42 KB
ptyprocess-0.6.

 ---> Running in 9ede72fc23c9
Removing intermediate container 9ede72fc23c9
 ---> 5fdc1f86222f
Step 11/11 : ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH
 ---> Running in acabb6ce3ec4
Removing intermediate container acabb6ce3ec4
 ---> 8755795cf873
Successfully built 8755795cf873
Successfully tagged georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.60
Sending build context to Docker daemon  4.608kB
Step 1/17 : FROM continuumio/miniconda3:4.7.10 as AzureMLSDKOnly
 ---> 4a51de2367be
Step 2/17 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> 74f737b6c8ba
Step 3/17 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gpg-agent     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> d1a7777a7a7a
Step 4/17 : ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
 ---> Using cache
 ---> 70d201fdbf5e
Step 5/

  3250K .......... .......... .......... .......... .......... 39% 3.27M 0s
  3300K .......... .......... .......... .......... .......... 40%  123M 0s
  3350K .......... .......... .......... .......... .......... 41%  167M 0s
  3400K .......... .......... .......... .......... .......... 41%  179M 0s
  3450K .......... .......... .......... .......... .......... 42%  150M 0s
  3500K .......... .......... .......... .......... .......... 42%  182M 0s
  3550K .......... .......... .......... .......... .......... 43%  173M 0s
  3600K .......... .......... .......... .......... .......... 44%  168M 0s
  3650K .......... .......... .......... .......... .......... 44%  203M 0s
  3700K .......... .......... .......... .......... .......... 45%  157M 0s
  3750K .......... .......... .......... .......... .......... 45%  195M 0s
  3800K .......... .......... .......... .......... .......... 46%  181M 0s
  3850K .......... .......... .......... .......... .......... 47%  168M 0s
  3900K ....

Removing intermediate container 8c3c650f1872
 ---> 73631c8be885
Step 16/17 : RUN rm -rf /tmp/azcopy
 ---> Running in f7c2e02b302c
Removing intermediate container f7c2e02b302c
 ---> 1726aa5f4432
Step 17/17 : ENV PATH=/azcopy10/azcopy_linux_amd64_10.1.2:$PATH
 ---> Running in 52b7ea620e4a
Removing intermediate container 52b7ea620e4a
 ---> 210a4af53378
Successfully built 210a4af53378
Successfully tagged georgedockeraccount/aml-sdk_docker_imageazcli:sdk.v1.0.60
Sending build context to Docker daemon  4.608kB
Step 1/20 : FROM continuumio/miniconda3:4.7.10 as AzureMLSDKOnly
 ---> 4a51de2367be
Step 2/20 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> 74f737b6c8ba
Step 3/20 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gpg-agent     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> d1a7777a7a7a
Step 4/20 :

In [10]:
# import docker

# cli = docker.APIClient()
# response=''
# response = [line for line in cli.build(
#     path=working_path, 
#     dockerfile=docker_file_path, 
#     pull=True,
#     rm=True, 
#     quiet=False,
#     nocache=True,
#     tag=aml_docker_image_name
# )]
# response[100:]

# client = docker.from_env()
# client.containers.list()

In [11]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir02']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02'

total 4
-rw-r--r-- 1 root root 7 Sep 11 03:24 exported_conda_env.yml


In [12]:
# export the resulted conda env

cli_command_inside_container = ' /bin/bash -c "conda env list ; pwd; ls -l /workspace ; ' + \
    'python -c \'import azureml.core;print(azureml.core.VERSION)\'; ' + \
    'conda env export > /workspace/exported_conda_env.yml"'

cli_command='docker run -it --rm  --name aml-sdk_docker_container ' + \
'-v ' + crt_local_dir + ':/workspace:rw ' + \
aml_docker_image_name + \
cli_command_inside_container

cli_command

if 1:
    ! $cli_command
else:
    client = docker.from_env()
    client.containers.run(aml_docker_image_name, 
                          runtime='',
                          remove=True,
                          volumes={'/workspace': {'bind': '/workspace', 'mode': 'rw'}},
                          working_dir='/',
                          command=cli_command_inside_container)

'docker run -it --rm  --name aml-sdk_docker_container -v /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.60 /bin/bash -c "conda env list ; pwd; ls -l /workspace ; python -c \'import azureml.core;print(azureml.core.VERSION)\'; conda env export > /workspace/exported_conda_env.yml"'

# conda environments:
#
base                     /opt/conda
aml-sdk-conda-env     *  /opt/conda/envs/aml-sdk-conda-env

/
total 4
-rw-r--r-- 1 root root 7 Sep 11 03:24 exported_conda_env.yml
1.0.60

# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/opt/conda/lib/python3.7/site-packages/conda/exceptions.py", line 1062, in __call__
        return func(*args, **kwargs)
      File "/opt/conda/lib/python3.7/site-packages/conda_env/cli/main.py", line 73, in do_call
        exit_code = getattr(module, func_name)(args, parser)
      File "/opt/conda/lib/python3.7/site-packages/conda_env/cli/main_export.py", line 94, in execute
        ignore_channels=args.ignore_channels)
      File "/opt/conda/lib/python3.7/site-packages/conda_env/env.py", line 96, in from_environment
        precs = tuple(PrefixGraph(pd.iter_records()).graph)
      File "/opt/conda/lib/python3.7/site-packages/conda/core/prefix_data.py", line 130, in iter_records

In [13]:
saved_conda_dependency_file = 'aml_sdk_conda_dep_file'+os.getenv('image_version')+'.yml'
exported_conda_dependency_file = 'aml_sdk_conda_dep_file_exported'+os.getenv('image_version')+'.yml'

!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/
!cp $conda_dependency_file_path ./docker_history/$saved_conda_dependency_file
!cp {os.path.join(*([crt_local_dir, 'exported_conda_env.yml']))} ./docker_history/$exported_conda_dependency_file



### To run aml SDK, ssh into a vm, run the command below and then go to: 
your_vm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [14]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $aml_docker_image_name /bin/bash -c '"jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.60 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root"


### Run short tests
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in plain python and in a notebook.
  
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [15]:
!echo docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"
!docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"

!docker run  $aml_azcli_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; azcopy -h"
!docker run  $aml_azcli_azdevops_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; az devops -h"


docker run georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.60 /bin/bash -c python -c 'import azureml.core;print(azureml.core.VERSION)'
1.0.60
1.0.60
azure-cli                         2.0.72 *

command-modules-nspkg               2.0.3
core                              2.0.72 *
nspkg                              3.0.4
telemetry                          1.0.3

Python location '/opt/az/bin/python3'
Extensions directory '/root/.azure/cliextensions'

Python (Linux) 3.6.5 (default, Aug 22 2019, 06:32:45) 
[GCC 6.3.0 20170516]

Legal docs and information: aka.ms/AzureCliLegal


/bin/bash: azcopy: command not found
1.0.60
azure-cli                         2.0.72 *

command-modules-nspkg               2.0.3
core                              2.0.72 *
nspkg                              3.0.4
telemetry                          1.0.3

Extensions:
azure-devops                      0.12.0

Python location '/opt/az/bin/python3'
Extensions directory '/root/.azure/cliextensions'

Python (Linux) 3.6.5 

#### Do the same in a  notebook

In [16]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [17]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02/test_aml_sdk_docker_image.ipynb


In [18]:
!ls -l $crt_local_dir

total 8
-rw-r--r-- 1 root       root         7 Sep 11 03:49 exported_conda_env.yml
-rw-rw-r-- 1 loginvm001 loginvm001 982 Sep 11 03:50 test_aml_sdk_docker_image.ipynb


#### Run the notebook and see the new SDK version


In [19]:
!docker run -v $crt_local_dir:/workspace:rw $aml_docker_image_name /bin/bash -c "jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 982 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [20]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 1.0.60\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [21]:
# !docker login -u=$docker_login -p=
!docker push $aml_docker_image_name
# !docker push $aml_sudo_docker_image_name
!docker push $aml_azcli_docker_image_name
!docker push $aml_azcli_azdevops_docker_image_name

The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_image]

59e2d5a8: Preparing 
c42b2518: Preparing 
1d870630: Preparing 
f8fc4c9a: Preparing 
ba47210e: Preparing 


59e2d5a8: Pushing  1.887GB/2.715GBAPushing  956.2MB/2.715GB

59e2d5a8: Pushed   2.772GB/2.715GBPushing  2.493GB/2.715GBsdk.v1.0.60: digest: sha256:84e8149c530413a00b8344794ae1ecf976df918412b054ced00998f4ad8d69a3 size: 1586
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcli]

73bfd1e5: Preparing 
cf40ae79: Preparing 
f5ebeb7e: Preparing 
59e2d5a8: Preparing 
c42b2518: Preparing 
1d870630: Preparing 
f8fc4c9a: Preparing 
ba47210e: Preparing 
f5ebeb7e: Pushed   470.3MB/439.4MBraccount/aml-sdk_docker_image Pushing  358.9MB/439.4MBsdk.v1.0.60: digest: sha256:535d06bcd48b3a45a946c30dbe8b1ad483a4bd684a5a93dbeb27378169e3f7ea size: 2218
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcliazdevops]

72180a8f: Preparing 
73bfd1e5: Preparing 
cf40ae79: Preparing 
f5ebeb7e: Preparing 
59e2d5a8: Preparing 
c42b2518: Preparing 
1d870630: Preparing 
f8fc4c9a: Preparing 
ba47210e: Preparing 
2180a8f: Pushed   17.08MB/15.69MBraccount/aml-sdk_docker_imageazcli sdk.v1.0.60: digest: sha256:5f1e35e

In [22]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker.ipynb to html
[NbConvertApp] Writing 521666 bytes to createAMLSDKDocker.html
